In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from matplotlib import pyplot as plt
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:
test = pd.read_csv('/kaggle/input/dont-overfit-ii/test.csv')
train = pd.read_csv('/kaggle/input/dont-overfit-ii/train.csv')

Ici nous donnons à nos tableaux de données un nom, "test" pour l'echantillon de test et "train" pour l'échantillon d'entrainement

In [ ]:
x_train = train.drop(["target","id"], axis = 1)
y_train = train["target"]
x_test = test.drop(["id"], axis = 1)

Nous associons à la variable x_train/x_test le tableau de données d'entrainement/test en enlevant les id et les données cibles.
Nous associons à la variable y_train/y_test les données cibles "target" d'entrainement/test

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

Les données sont centrées et réduites

In [ ]:
pca = PCA(n_components = 75)
pca.fit(x_train)
x_train = pd.DataFrame(pca.transform(x_train))
x_test = pd.DataFrame(pca.transform(x_test))

Ici nous appliquons la PCA à notre ensemble de données d'entrainement. On peut choisir la dimension que l'on souhaite atteindre grace à (n_components = K) ou bien on peut simplement préciser la variance que l'on souhaite conserver avec notre réduction. (ici 60% de variance expliquée -> 0.6 ou 75 colonnes -> n_components = 75)

In [ ]:
ridge = linear_model.Ridge()
lasso = linear_model.Lasso()
elastic = linear_model.ElasticNet()
lasso_lars = linear_model.LassoLars()
bayesian_ridge = linear_model.BayesianRidge()
logreg = linear_model.LogisticRegression(solver='liblinear')
sgd = linear_model.SGDClassifier()
knc = KNeighborsClassifier(n_neighbors=20)
forest_reg = RandomForestRegressor(n_estimators = 10, random_state = 0)
models = [ridge, lasso, elastic, lasso_lars, bayesian_ridge, logreg, sgd, knc,forest_reg]

Ici nous déclarons tous les modèles 

In [ ]:
def cv_scores(model):
    scores = cross_val_score(model, x_train, y_train, cv=5, scoring='roc_auc')
    print('Moyenne des scores: ', np.mean(scores))
    print('Ecart-type des scores: ', np.std(scores))
    print('\n')

On crée une fonction qui va faire tourner tous les modèles un à un avec validation croisée (10 groupes), et qui va nous montrer la moyenne des scores et l'ecart type des scores de chaque modèle (inspiré du code d'un autre participant)

In [ ]:
for model in models:
    print(model)
    cv_scores(model)

Après avoir fait tourner la fonction, on remarque que le modèle ls plus performant est la regression logistique (moyenne des scores la plus élevée)

Maintenant nous allons chercher les paramètres qui permettent de maximiser les performances de la regression logistique 

In [ ]:
penalite = ['l1', 'l2']
C = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
class_weight = [{1:0.5, 0:0.5}, {1:0.4, 0:0.6}, {1:0.6, 0:0.4}, {1:0.7, 0:0.3}]
solver = ['liblinear', 'saga']

param_grid = dict(penalty=penalite,
                  C=C,
                  class_weight=class_weight,
                  solver=solver)

grid = GridSearchCV(estimator=logreg, param_grid=param_grid, scoring='roc_auc', verbose=1, n_jobs=-1)
grid_result = grid.fit(x_train, y_train)

print('Meilleur Score: ', grid_result.best_score_)
print('Meilleurs Paramètres: ', grid_result.best_params_)

Ici on a obtenu les meilleurs paramètres pour la regression logistique

In [ ]:
logreg = linear_model.LogisticRegression(C=0.001, class_weight={1:0.4, 0:0.6}, penalty='l2', solver='liblinear')
cv_scores(logreg)

In [ ]:
y_test = logreg.fit(x_train, y_train).predict_proba(x_test)